In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import string

from string import punctuation
from os import listdir
from collections import Counter

from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Activation

import os
import csv
import pickle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#getting the y values for train-test

y_col_list = ["2cl.undir"]
y = pd.read_csv("fanfic-corpus.csv", usecols=y_col_list)
print(y.loc[y['2cl.undir'] == 1])

num_of_pos = len((y.loc[y['2cl.undir'] == 1]).index)
num_of_neg = len((y.loc[y['2cl.undir'] == 0]).index)

print(num_of_pos)

print(num_of_neg)

print(num_of_pos / (num_of_pos + num_of_neg))

#df.loc[df['2cl.undir'] == '0']

#df.loc[df['column_name'] == some_value]

      2cl.undir
0             1
1             1
2             1
3             1
4             1
...         ...
1737          1
1738          1
1739          1
1740          1
1741          1

[1042 rows x 1 columns]
1042
700
0.5981630309988519


In [4]:
#getting only the fanfic text from document

col_list=["text"]
df = pd.read_csv("fanfic-corpus.csv", usecols=col_list)
full_df = pd.read_csv("fanfic-corpus.csv")
full_df.info()
df.to_csv("text_only.csv", header=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1742 entries, 0 to 1741
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  1742 non-null   int64 
 1   url         1742 non-null   object
 2   8cl.undir   1742 non-null   int64 
 3   8cl.dir     1742 non-null   int64 
 4   5cl.undir   1742 non-null   int64 
 5   5cl.dir     1742 non-null   int64 
 6   2cl.undir   1742 non-null   int64 
 7   2cl.dir     1742 non-null   int64 
 8   type        1742 non-null   object
 9   text        1742 non-null   object
dtypes: int64(7), object(3)
memory usage: 136.2+ KB


In [6]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	lines = file.read()
	# close the file
	file.close()
	return lines
 
# turn a doc into clean tokens
def clean_text(text):
	# split into tokens by white space
	tokens = text.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	#convert to lowercase
	for i in range(len(tokens)):
		tokens[i] = tokens[i].lower()
	
	return tokens
 
# load the document
filename = 'text_only.csv'
#for i in range(1742):
#	text = df[i]
text = load_doc(filename)
tokens = clean_text(text)
print(tokens)

['likely', 'wont', 'bit', 'pissed', 'simon', 'sighed', 'he', 'loved', 'clary', 'really', 'sometimes', 'wondered', 'much', 'easier', 'lifed', 'hed', 'never', 'friends', 'which', 'treacherous', 'thing', 'even', 'casually', 'think', 'couldnt', 'help', 'instead', 'calling', 'nearly', 'comatose', 'vampires', 'couldve', 'playing', 'playstation', 'okay', 'try', 'but', 'cant', 'promise', 'hell', 'wake', 'much', 'less', 'willingly', 'help', 'targjacetarg', 'guys', 'expraphaelexp', 'which', 'understatement', 'year', 'unless', 'simons', 'well', 'raphael', 'least', 'helpful', 'person', 'knew', 'kind', 'sweet', 'really', 'simon', 'shook', 'head', 'time', 'place', 'he', 'fished', 'phone', 'pocket', 'dialing', 'raphaels', 'number', 'shadowhunters', 'watched', 'simon', 'frowned', 'why', 'cant', 'alec', 'call', 'boyfriend', 'way', 'im', 'sure', 'magnus', 'knows', 'lot', 'things', 'asked', 'waited', 'raphael', 'dont', 'think', 'realize', 'kind', 'schedules', 'vampires', 'keep', 'everyone', 'present', 'c

In [7]:
# Read by lines.

def load_lines(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	lines = file.readlines()
	# close the file
	file.close()
	return lines

# load doc and add to vocab
def add_doc_to_vocab(filename, vocab):
	# load doc
	doc = load_doc(filename)
	# clean doc
	tokens = clean_text(doc)
	# update counts
	vocab.update(tokens)

def csv_to_lines(filename, vocab):
	lines = load_lines(filename)
	
	clean_lines = [clean_text(line) for line in lines]
	
	return clean_lines

def process_docs(filename, vocab):
		# add doc to vocab
		add_doc_to_vocab(filename, vocab)

# define vocab
vocab = Counter()


# add all docs to vocab

lines = csv_to_lines('text_only.csv', vocab)
process_docs('text_only.csv', vocab)

# print the size of the vocab
print(len(vocab))

# print the top words in the vocab
print(vocab.most_common(50))
print(lines)

4579
[('he', 2195), ('said', 1257), ('she', 1070), ('the', 949), ('back', 846), ('like', 807), ('eyes', 791), ('it', 727), ('you', 721), ('would', 677), ('could', 655), ('know', 636), ('one', 622), ('even', 546), ('stiles', 524), ('love', 521), ('jacob', 518), ('time', 510), ('draco', 492), ('head', 482), ('looked', 479), ('something', 470), ('little', 465), ('tony', 461), ('two', 448), ('yuugi', 445), ('what', 443), ('man', 439), ('harry', 437), ('well', 433), ('face', 428), ('get', 426), ('hermione', 417), ('simon', 416), ('and', 415), ('way', 410), ('atemu', 405), ('asked', 403), ('going', 401), ('look', 382), ('think', 379), ('good', 379), ('but', 377), ('we', 376), ('around', 351), ('see', 350), ('tell', 344), ('got', 341), ('bucky', 340), ('really', 337)]
[['likely', 'wont', 'bit', 'pissed', 'simon', 'sighed', 'he', 'loved', 'clary', 'really', 'sometimes', 'wondered', 'much', 'easier', 'lifed', 'hed', 'never', 'friends', 'which', 'treacherous', 'thing', 'even', 'casually', 'think

In [8]:
# save list to file
def save_list(lines, filename):
	# convert lines to a single blob of text
	data = '\n'.join(lines)
	# open file
	file = open(filename, 'w')
	# write text
	file.write(data)
	# close file
	file.close()
 
# save tokens to a vocabulary file
print(vocab)
print(vocab.keys())
print(vocab["text"])
print(len(vocab.keys()))

save_list(vocab.keys(), 'vocab.txt')
print(vocab.keys())

Counter({'he': 2195, 'said': 1257, 'she': 1070, 'the': 949, 'back': 846, 'like': 807, 'eyes': 791, 'it': 727, 'you': 721, 'would': 677, 'could': 655, 'know': 636, 'one': 622, 'even': 546, 'stiles': 524, 'love': 521, 'jacob': 518, 'time': 510, 'draco': 492, 'head': 482, 'looked': 479, 'something': 470, 'little': 465, 'tony': 461, 'two': 448, 'yuugi': 445, 'what': 443, 'man': 439, 'harry': 437, 'well': 433, 'face': 428, 'get': 426, 'hermione': 417, 'simon': 416, 'and': 415, 'way': 410, 'atemu': 405, 'asked': 403, 'going': 401, 'look': 382, 'think': 379, 'good': 379, 'but': 377, 'we': 376, 'around': 351, 'see': 350, 'tell': 344, 'got': 341, 'bucky': 340, 'really': 337, 'want': 336, 'voice': 334, 'much': 331, 'right': 326, 'mr': 324, 'they': 320, 'hair': 320, 'keith': 320, 'never': 318, 'pack': 315, 'smiled': 308, 'says': 306, 'knew': 304, 'his': 297, 'hand': 296, 'say': 295, 'turned': 295, 'room': 293, 'still': 289, 'need': 288, 'help': 286, 'us': 284, 'scott': 281, 'came': 278, 'made': 2

In [9]:
# create the tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()

# fit the tokenizer on the documents
tokenizer.fit_on_texts(vocab.keys())

In [10]:
#split train-test data

# select a column as series and then convert it into a column
list_df = df['text'].to_list()
list_y = y['2cl.undir'].to_list()

xTrainLines, xTestLines, yTrain, yTest = train_test_split(list_df, list_y, test_size=0.1)
print(len(xTrainLines), len(yTrain))
print(len(y))
print(type(xTrainLines))

1567 1567
1742
<class 'list'>


In [11]:
# encode training data set
xTrain = tokenizer.texts_to_matrix(xTrainLines, mode='freq')
print(xTrain.shape)
print(xTrain[0][:20])
print(lines[0])

xTest = tokenizer.texts_to_matrix(xTestLines, mode='freq')
print(xTest.shape)

(1567, 4580)
[0.         0.         0.         0.         0.         0.
 0.         0.00714286 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
['likely', 'wont', 'bit', 'pissed', 'simon', 'sighed', 'he', 'loved', 'clary', 'really', 'sometimes', 'wondered', 'much', 'easier', 'lifed', 'hed', 'never', 'friends', 'which', 'treacherous', 'thing', 'even', 'casually', 'think', 'couldnt', 'help', 'instead', 'calling', 'nearly', 'comatose', 'vampires', 'couldve', 'playing', 'playstation', 'okay', 'try', 'but', 'cant', 'promise', 'hell', 'wake', 'much', 'less', 'willingly', 'help', 'targjacetarg', 'guys', 'expraphaelexp', 'which', 'understatement', 'year', 'unless', 'simons', 'well', 'raphael', 'least', 'helpful', 'person', 'knew', 'kind', 'sweet', 'really', 'simon', 'shook', 'head', 'time', 'place', 'he', 'fished', 'phone', 'pocket', 'dialing', 'raphaels', 'number', 'shadowhunters', 'watched', 'simon', 'frowned', 'why', 'can

In [12]:
# saving xTrainLines
#rows = zip(xTrainLines)

# with open('xTrainLines.csv','w') as result_file:
#     wr = csv.writer(result_file)
#     for row in rows:
#         wr.writerow(row)

with open('xTrainLines.txt', 'w') as f:
    for item in xTrainLines:
        f.write("%s\n" % item)

# saving xTestLines
# rows = zip(xTestLines)

# with open('xTestLines.csv','w') as result_file:
#     wr = csv.writer(result_file)
#     for row in rows:
#         wr.writerow(row)

with open('xTestLines.txt', 'w') as f:
    for item in xTestLines:
        f.write("%s\n" % item)

# saving yTrain
# rows = zip(yTrain)

# with open('yTrain.csv','w') as result_file:
#     wr = csv.writer(result_file)
#     for row in rows:
#         wr.writerow(row)

with open('yTrain.txt', 'w') as f:
    for item in yTrain:
        f.write("%s\n" % item)

# saving yTest
# rows = zip(yTest)

# with open('yTest.csv','w') as result_file:
#     wr = csv.writer(result_file)
#     for row in rows:
#         wr.writerow(row)

with open('yTest.txt', 'w') as f:
    for item in yTest:
        f.write("%s\n" % item)

In [13]:
model = load_model('path')

In [14]:
# classify a text as negative (0) or positivte (1)
def predict_sentiment(review, vocab, tokenizer, model):
	# clean
	tokens = clean_text(review)
	# filter by vocab
	tokens = [w for w in tokens if w in vocab]
	# convert to line
	line = ' '.join(tokens)
	# encode
	encoded = tokenizer.texts_to_matrix([line], mode='freq')
	# prediction
	yhat = model.predict(encoded, verbose=0)
	return round(yhat[0,0])

In [16]:
text1 = "'Xiao Xingchen? Are you alright?' Xue Yang said anxiously as he saw the daozhang walking towards him. 'Yes, I am.' 'Oh, that's good to hear. Don't get hurt. A-Qing will never let me hear the end of it. Stay safe, alright?' 'Of course. I'm sorry that you have to worry about me. 'Oh, it's fine. I just want you to be safe, you know?'"
#print(predict_sentiment(text1, vocab, tokenizer, model))
# # test negative text
text2 = "'Xue Yang!' Song Lan's angry voice echoed across the empty village. Xue Yang grinned and shrugged casually. 'What, I didn't hurt him! Why are you so angry?' 'You tricked him! You fooled him! I can't stand you, you criminal!' 'Well I hate you too,' Xue Yang sighed, giving him a glare before turning away. 'I don't understand why someone so terrible like you would want to stick around with my precious Daozhang. Have you ever helped him? No. I did more than you ever will!' 'Shut up!'"
#print(predict_sentiment(text2, vocab, tokenizer, model))

text3 = "'Hinata!' We're going to be late!' Komaeda called. 'Coming!' Hinata yelled, and he grabbed a slice of toast as he ran out the door. 'Thanks for reminding me,' he said breathlessly before taking a bite of toast. 'No problem,' Komaeda said, laughing. 'It's what friends do, no?' He began skipping down the sidewalk, singing out, 'Last day of school! Last do of school! Aren't you excited, Hinata?' 'Kind of,' his friend replied. 'I just don't know what I'll do this summer. 'We can go to the fairs together! The pool!' 'I never win anything at fairs...' 'Well, with my luck, I'll help you win!' 'Uh, Komaeda, isn't that cheating?' 'But we win, don't we?' 'Yeah, I guess we do...'"
#print(predict_sentiment(text3, vocab, tokenizer, model))

text_and_dropdown = "'Xue Yang!' Song Lan's angry voice echoed across the empty village. Xue Yang grinned and shrugged casually. 'What, I didn't hurt him! Why are you so angry?' 'You tricked him! You fooled him! I can't stand you, you criminal!'" #@param ["'Xiao Xingchen? Are you alright?' Xue Yang said anxiously as he saw the daozhang walking towards him. 'Yes, I am.' 'Oh, that's good to hear. Don't get hurt. A-Qing will never let me hear the end of it. Stay safe, alright?' 'Of course. I'm sorry that you have to worry about me. 'Oh, it's fine. I just want you to be safe, you know?'", "'Xue Yang!' Song Lan's angry voice echoed across the empty village. Xue Yang grinned and shrugged casually. 'What, I didn't hurt him! Why are you so angry?' 'You tricked him! You fooled him! I can't stand you, you criminal!'", "'Hinata!' We're going to be late!' Komaeda called. 'Coming!' Hinata yelled, and he grabbed a slice of toast as he ran out the door. 'Thanks for reminding me,' he said breathlessly before taking a bite of toast. 'No problem,' Komaeda said, laughing. 'It's what friends do, no?' He began skipping down the sidewalk, singing out, 'Last day of school! Last do of school! Aren't you excited, Hinata?' 'Kind of,' his friend replied. 'I just don't know what I'll do this summer. 'We can go to the fairs together! The pool!' 'I never win anything at fairs...' 'Well, with my luck, I'll help you win!' 'Uh, Komaeda, isn't that cheating?' 'But we win, don't we?' 'Yeah, I guess we do...'"] {allow-input: true}
prediction = predict_sentiment(text_and_dropdown, vocab, tokenizer, model)

if (prediction == 0):
    print("Negative.")
    print(prediction)
else:
    print("Positive.")
    print(prediction)


Negative.
0.0


In [17]:
# #rows = zip(vocab)

# with open('vocabCount.csv', encoding='utf-8-sig', mode='w') as result_file:
#   writer=csv.writer(result_file)
#   for key, value in vocab.items():
#     #for row in rows:
#     writer.writerow([key + ": " + str(value) + '\n'])  